In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Importing the libraries 
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import matplotlib.cm as cm
import pandas as pd
import librosa
import librosa.display
import torch.nn.functional as F
from torch.autograd import Variable
import time
from Models import Autoencoder
from TestData import SingleMus

# Checking the availability of CUDA on machine
print("Torch", torch.__version__, "CUDA", torch.version.cuda)
print("Device:", torch.device("cuda:0"))
print(torch.cuda.is_available())



In [ ]:
# label_dict = {
#     'P': 0,
#     'R': 1,
#     'J': 2,
#     'G': 3,
#     'B': 4,
#     'C': 5,
#     'A': 6,
#     'H': 7,
#     'K': 8,
#     'E': 9,
#     'S': 10,
#     'T': 11,
#     'L': 12}

In [ ]:
filePath = '/media/sdc1/Data/ETRI_Music'
muspath_list = [i for i in os.listdir(filePath) if i.endswith('.wav')]
# label_list = [label_dict.get(i[-7]) for i in muspath_list]


In [ ]:
# for i in range(13):
#     count = sum(np.array(label_list)==i)
#     print(i, count)

In [ ]:
# # 1/5 test data; 4/5 training data
# train_index = []
# test_index = []
# for genre in range(13):
#     index = [i for i in range(len(label_list)) if label_list[i] == genre]
#     perm_index = np.random.permutation(index).tolist()
#     test_index += perm_index[:int(len(index)/4)]
#     train_index += perm_index[int(len(index)/4):]

# test_index = np.random.permutation(test_index).tolist()
# train_index = np.random.permutation(train_index).tolist()

# train_path = [muspath_list[i] for i in train_index]
# test_path = [muspath_list[i] for i in test_index]
# label_tr = [label_list[i] for i in train_index]
# label_te = [label_list[i] for i in test_index]


In [ ]:
# # dataloading
# error_file = []

# trx, tex = [], []
# i = 0
# for path in train_path:
#     print(i)
#     i+=1
#     try:
#         s, sr=librosa.load(filePath+'/'+path, sr=None) 
#         trx.append(list(s/np.std(s)))
#     except:
#         print(i, path)
#         print(label_tr.pop(i))
#         error_file.append(path)
#         continue
    
# i = 0
# for path in test_path:
#     i+=1
#     try:
#         s, sr=librosa.load(filePath+'/'+path, sr=None) 
#         tex.append(s/np.std(s))
#     except:
#         print(i, path)
#         print(label_te.pop(i))
#         error_file.append(path)
#         continue
    
# # 


In [ ]:
# maxAmp=0
# for signal in trx:
#     if np.max(np.abs(signal))>maxAmp:
#         maxAmp=np.max(np.abs(signal))

# maxAmp*=1.5

# trx /= maxAmp
# tex /= maxAmp


In [ ]:
# trx_c = []
# tex_c = []
# for i in range(len(trx)):
# #     print(i)
#     trx_c.append(trx[i][:1024])
# for i in range(len(tex)):
# #     print(i)
#     tex_c.append(tex[i][:1024])

In [ ]:
# trx_c = np.array([np.array(_) for _ in trx_c])
# tex_c = np.array([np.array(_) for _ in tex_c])

In [ ]:
# Testing the trained model
def testClassifier():
    correct = 0
    total = 0
    save_one = 0
    bs = 30
    with torch.no_grad():
        for i in range(0, len(tex_c), bs):
            end = np.minimum(i+bs, len(tex_c))
            data = torch.FloatTensor(np.asarray(tex_c[i:end])) 
            labels = torch.FloatTensor(np.asarray(label_te[i:end]))

            outputs = model(data)
            #print(outputs.data)
            try:
                maxdata, predicted = torch.max(outputs.data, 1)
            except:
                print(predicted)
            total += labels.size(0)
            try:
                correct += (predicted == labels.long()).sum().item()
            except:
                print(predicted.shape)
                print(labels.shape)
                fake_function()

    #print(correct)
    print("Accuracy:" , '{:.2f}'.format(100.0 * correct / total))


In [ ]:
## Training classifier

model = musClassifier()

lr = 0.001
optimizer= torch.optim.Adam(model.parameters(), lr)#, betas=[0.9, 0.999])
loss_function = nn.CrossEntropyLoss()

epochs = 50
bs = 1024

for t in range(epochs):
    start = time.time()
    for i in range(0, len(trx_c), bs):
        end = np.minimum(i+bs, len(trx_c))
        data = torch.FloatTensor(np.asarray(trx_c[i:end]))     
        labels = torch.IntTensor(np.asarray(label_tr[i:end])) 

        optimizer.zero_grad()
        outputs = model(data)
#         print(outputs)
        loss = loss_function(outputs, labels.long())
        loss.backward()
        optimizer.step()
    
    print('epoch_{}'.format(t))
    testClassifier()
    end = time.time()
    print('loss:' '{:.3f}'.format(loss.data.cpu().numpy()))
#           , '{:.0f}'.format(end-start))
